# Patch extraction from Histology Images
<a href="https://colab.research.google.com/github/TIA-Lab/tiatoolbox/blob/master/examples/example_patchextraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## About this notebook
This  jupyter notebook can be run on any computer with a standard browser and no programming language is required. It can run remotely over the Internet, free of charge, thanks to Google Colaboratory. To connect with Colab, click on the badge "Open in Colab" above. Familiarize yourself with the drop-down menus near the top of the window. You can edit the notebook during the session, for example substituting your own image files for the image files used in this demo. Experiment by changing the parameters of functions. It is not possible for an ordinary user to permanently change this version of the notebook on either Github or colab, so you cannot inadvertently mess it up, and you should feel free to experiment. Use the notebook's File Menu if you wish to save your own (changed) notebook.

Before running the notebook outside Colab, set up your Python environment, as explained in the 
[README](https://github.com/TIA-Lab/tiatoolbox/blob/develop/README.md#install-python-package) file.

## Welcome to Tiatoolbox
In this example we will show how you can use tiatoolbox to extract patches from a large histology image. Tiatoolbox can extract patches in different ways, such as point-based, fixed-window, and variable-window patch extraction. One practical use of these tools is when using deep learning models that cannot accept large images in the input. In particular, we will introduce the use of our module
`patchextraction` ([details](https://github.com/TIA-Lab/tiatoolbox/blob/develop/tiatoolbox/tools/patchextraction.py)).

### First cell in bash
This cell prepares the Colab environment for the use of `tiatoolbox`. The first line of code in this cell indicates that the rest of the cell will be interpreted in bash. The other lines set up a suitable Python environment under Colab, and produce no output if the cell has been run before in the same Colab session.

In [ ]:
%%bash
if [ $(env | grep COLAB | wc -c) -gt 0 ] && [ $( pip list | grep datascience | wc -c) -gt 0 ]
then
    echo "uninstalling colab packages with conflicts"
    pip uninstall -y datascience
    pip uninstall -y albumentations
    apt-get -y install libopenjp2-7-dev libopenjp2-tools openslide-tools
    pip install git+git://github.com/TIA-Lab/tiatoolbox@develop
fi

## Importing related libraries
We will start by importing some libraries required to run this notebook.

In [ ]:
from tiatoolbox.tools import patchextraction
from tiatoolbox.utils.misc import imread
from tiatoolbox.utils.misc import read_locations
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 300 # for high resolution figure in notebook

### Downloading the required files
We download, over the internet, a couple of files (a histology image and a csv file containing the positions of nuclei in that image). Download is needed once in each Colab session.

In [ ]:
import requests

img_file_name = "sample_img.png"
csv_file_name = "sample_coordinates.csv"

# Downloading sample image
r = requests.get("https://tiatoolbox.dcs.warwick.ac.uk/testdata/patchextraction/TCGA-HE-7130-01Z-00-DX1.png")
with open(img_file_name, "wb") as f:
    f.write(r.content)

# Downloading points list 
r = requests.get("https://tiatoolbox.dcs.warwick.ac.uk/testdata/patchextraction/sample_patch_extraction.csv")
with open(csv_file_name, "wb") as f:
    f.write(r.content)

# Reading image and annotation file
We use a sample image from the [MoNuSeg](https://monuseg.grand-challenge.org/Data/) dataset, for which nuclei have already been located (manually) and centroids computed. The sample image and list of points are loaded from the internet. The function `read_locations` returns a dataFrame, in which a typical row has the form $(x, y, class)$. Here $(x,y)$  are coordinates for a particular centroid, and *class* is the type of that patch. For this example, "class" can indicate the type of the nucleus, such as "epithelial" or "inflammatory". In the simple situation we are illustrating here, biological information has not been provided, and is replaced by a meaningless number 0.0, which is just a place-holder. Dataframes in Python are handled using `pandas`--- you don't need to learn the details to understand this demo, but, if you want to use dataframes in your own code, or to replace data in this notebook by your own data, you will need to find out more about pandas, for example from Google.

In [ ]:
input_img = imread(img_file_name)
centroids_list = read_locations(csv_file_name) 

print('Image size: {}'.format(input_img.shape))
print('This image has {} point annotations'.format(
                                        centroids_list.shape[0]))
print('First few lines of dataframe:\n', centroids_list.head())

To see better what we are dealing with here, we show the image, first in its original form, and then with the desired centroids overlaid.

In [ ]:

    
input_img = imread(img_file_name)
plt.imshow(input_img)
plt.axis('off')
plt.show()

# overlay nuclei centroids on image and plot
plt.imshow(input_img)
plt.scatter(np.array(centroids_list)[:,0], np.array(centroids_list)[:,1], s=1)
plt.axis('off')
plt.show()

## Patches based on point annotations
As you can see in the above figure, each nucleus is marked with a blue dot. To train a nucleus classifier computer program (or a beginning pathologist), it is helpful to see a nucleus in context, that is, within a. surrounding patch. We therefore extract, for each nucleus, a patch centred on that nucleus. If the third column of our dataframe has been completed meaningfully (which is not the case in our example), it is then easy to save patches in different folders based on their biological significance or class=classification. This can be done using functions from the Python classes defined in our module `patchextraction` . (We are using both class=biological classification and class=Python class for coding.) The `patch_extractor` yields patches from the image, `input_img`, based on the `centroids_list` in a one-by-one manner. In the next code cell, we show how to use the function `get_patch_extractor` to obtain a suitable `patch_extractor`

In [ ]:
patch_extractor = patchextraction.get_patch_extractor(
        input_img=input_img, # input image path, numpy array, or WSI object
        locations_list=np.array(centroids_list)[300:350,:], # path to list of points (csv, json), numpy list, panda DF
        method_name="point", # also supports "fixedwindow" and "variablewindow"
        patch_size=(32, 32), # size of the patch to extract around the centroids from centroids_list
        resolution=0,
        units="level",
    )

As you can see, `patchextraction.get_patch_extractor` accepts several arguments:

- `input_img`: The image from which we want to extract patches. We first read the image and pass it to the function. Instead, you can pass the path of the image to the function.
- `locations_list`: The list of points at which the required patches will be centred. We load the points list as a panda data frame and pass it to the function. Instead, you can pass to the function the path to a csv or json file.
- `method_name`: This important argument specifies the type of patch extractor that we want to build. As we are looking to extract patches around centroid points, we use here the `point` option. Other options, for example `fixedwindow` and `variablewindow`, are also supported. Please refer to the documentation for more information.
- `patch_size`: Size of the patches. 
- `resolution` and `unit`: These arguments specify the level or micron-per-pixel resolution of the WSI. Here we specify the WSI's level 0. In general, this is the level of greatest resolution, although, in this particular case, the image has only one level.

The `patch_extractor` yields information in small chunks, so as to avoid potential memory problems when the list of centroids is very long.
To extract patches using the `patch_extractor` we use ***for loop***s as below, where we extract the first 16 patches specified by `centroids_list`.

In [ ]:
i = 1
for patch in patch_extractor:
    plt.subplot(4,4,i)
    plt.imshow(patch)
    plt.axis('off')
    if i >= 16: # show only first 16 patches
        break
    i += 1
plt.show()


# Generate fixed-size patches
A very common practice in computational pathology, when analyzing large histology images or WSIs, is to extract overlapping patches from that image and analyse them one-by-one. Deep Learning models often cannot accept large images due to memory limitations. We designed a tool in Tiatoolbox to ease the process of overlapping patch extraction for such goals. 

The same `patchextraction` class supports another method that allows the user to extract all the patches from the input image in an efficient way, using just one line of code. In order to do that, one changes the method name in the `patchextraction` to `"fixedwindow"` as below: 


In [ ]:
fixed_patch_extractor = patchextraction.get_patch_extractor(
        input_img=input_img, # input image path, numpy array, or WSI object
        method_name="fixedwindow", # also supports "fixedwindow" and "variablewindow"
        patch_size=(500, 500), # size of the patch to extract around the centroids from centroids_list
        stride=(500, 500) # stride of extracting patches, default is equal to patch_size
    )

The `patchextraction` splits the input image into patches of size 500x500 without any overlap, because the `stride` of patch extraction is the same as `patch_size`. The `fixed_patch_extractor` is an iterator that yields a patch each time it is called. As in the example above, we can use a **for** loop to access these patches:

In [ ]:
i = 1
for patch in fixed_patch_extractor:
    plt.subplot(2,2,i)
    plt.imshow(patch)
    plt.axis('off')
    i += 1
plt.show()

Otherwise, by setting the `stride` smaller than the 'patch_size`, we can extract overlapping patches. Below we extract 500x500 patches that have 250 pixels overlap in both axes. 

In [ ]:
fixed_patch_extractor = patchextraction.get_patch_extractor(
        input_img=input_img, # input image path, numpy array, or WSI object
        method_name="fixedwindow", # also supports "fixedwindow" and "variablewindow"
        patch_size=(500, 500), # size of the patch to extract around the centroids from centroids_list
        stride=(250, 250) # 250 pixels overlap in both axes
    )

i = 1
for patch in fixed_patch_extractor:
    plt.subplot(3,3,i)
    plt.imshow(patch)
    plt.axis('off')
    i += 1
plt.show()